In [17]:
import pandas as pd
%matplotlib inline

Practice Question 1: What's the size of this dataset? (i.e., how many trips)

In [1]:
! bq query --use_legacy_sql=false 'SELECT COUNT(*) FROM `bigquery-public-data.san_francisco.bikeshare_trips`'

Waiting on bqjob_r5514a004f3d24ab5_00000168bbf5fee0_1 ... (0s) Current status: DONE   
+--------+
|  f0_   |
+--------+
| 983648 |
+--------+


Practice Question 2: What is the earliest start time and latest end time for a trip?

In [19]:
! bq query --use_legacy_sql=false '\
SELECT EXTRACT(TIME FROM min(start_date)), EXTRACT (TIME FROM max(end_date)) \
FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
'

Waiting on bqjob_r7dedd6106f5b5c0c_00000168a0c5bf29_1 ... (0s) Current status: DONE   
+----------+----------+
|   f0_    |   f1_    |
+----------+----------+
| 09:08:00 | 23:48:00 |
+----------+----------+


Practice question 3: How many bikes are there?

In [20]:
! bq query --use_legacy_sql=false ' \
SELECT COUNT(distinct bike_number) \
FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
'

Waiting on bqjob_r3693cd955572f2ad_00000168a0c5ca92_1 ... (0s) Current status: DONE   
+-----+
| f0_ |
+-----+
| 700 |
+-----+


New query : How many trips are in the morning vs in the afternoon

In [21]:
! bq query --use_legacy_sql=false ' \
SELECT COUNT(*) \
FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
WHERE EXTRACT(TIME FROM start_date) < "12:00:00" \
'

Waiting on bqjob_r48f5c99e4e8cb9a7_00000168a0c5d801_1 ... (0s) Current status: DONE   
+--------+
|  f0_   |
+--------+
| 412339 |
+--------+


In [23]:
! bq query --use_legacy_sql=false '\
SELECT COUNT(*) \
FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
WHERE EXTRACT(TIME FROM start_date) > "12:00:01" \
'

Waiting on bqjob_r40ac0af832ad847b_00000168a11306d9_1 ... (0s) Current status: DONE   
+--------+
|  f0_   |
+--------+
| 570556 |
+--------+


Answer:

Morning trips: 412339

Afternoon trips: 570556

### Project Questions

Identify the main questions you'll need to answer to make recommendations (list
below, add as many questions as you need).

- Question 1: Are more revenues coming from weekend vs weekdays? Which one should Ford incentivise?

- Question 2: What are the top 10 commutes on a weekday? (Advertize the top 5. Provide extra incentives for the next 5)

- Question 3: What are the top 10 commutes over the weekend? (Advertize the top 5. Monetize the first 5 some more (tournaments?))

- Question 4: Do subscribers use the bikeshare system more frequently on weekdays compared to customers? How about the weekend?

- Question 5: Do customers/subscribers take longer trips over the weekend? (This is related to providing incentives over the weekend)

- Question 6: Which docking stations are poorly utilized? Provide extra incentives to customers to use it: Ex: bike angel points

- Question 7: What times of the day are bikes used the least? Can Ford provide day of time based incentives to improve revenues?

- Question 8: Are more bikes in use over weekends and holidays? Are they used longer in weekends and holidays? Can Ford Bikeshare incentivise customers to prebook on weekends and holidays?

- Question 9: If Question 8 is true, which are the top 5 routes for which you can offer incentives?

- Question 10: Which zipcodes are the most frequently used? How can Ford Bikeshare incentivise the ones that are less frequently used based on customer demographics?

Before using the dataset, I did a small amount of cleansing to remove trips that are less than 2 minutes and greater than 10 hours. I used this cleaned up data set for the queries below

In [46]:
! bq query --use_legacy_sql=false '\
SELECT * FROM `bigquery-public-data.san_francisco.bikeshare_trips` \
WHERE NOT (duration_sec > 20*3600 OR duration_sec  < 2 *  60) \
LIMIT 5 \
'

Waiting on bqjob_r21fb40de1c7b87cb_00000168a411bc69_1 ... (0s) Current status: DONE   
+---------+--------------+---------------------+--------------------+------------------+---------------------+------------------+----------------+-------------+----------+-----------------+
| trip_id | duration_sec |     start_date      | start_station_name | start_station_id |      end_date       | end_station_name | end_station_id | bike_number | zip_code | subscriber_type |
+---------+--------------+---------------------+--------------------+------------------+---------------------+------------------+----------------+-------------+----------+-----------------+
|  944732 |         2618 | 2015-09-24 17:22:00 | Mezes              |               83 | 2015-09-24 18:06:00 | Mezes            |             83 |         653 | 94063    | Customer        |
|  984595 |         5957 | 2015-10-25 18:12:00 | Mezes              |               83 | 2015-10-25 19:51:00 | Mezes            |             83 |       

This was saved as reliable-jet-228900.bike_trip_data.bikeshare_trips_clean. The reduced dataset had 975375 rows (down from 983648)

In [43]:
! bq query --use_legacy_sql=false ' \
SELECT COUNT(*) \
FROM `reliable-jet-228900.bike_trip_data.bikeshare_trips_clean` \
'

Waiting on bqjob_r1a8e9ef7f8628db8_00000168a40e860e_1 ... (0s) Current status: DONE   
+--------+
|  f0_   |
+--------+
| 975375 |
+--------+


#### Question 1: 
Are more revenues coming from weekend vs weekdays? Which one should Ford incentivise?

In [26]:
! bq query --use_legacy_sql=false ' \
SELECT EXTRACT (DAYOFWEEK from start_date) as day_of_week, SUM(duration_sec) as trip_length_sum_per_week \
FROM `reliable-jet-228900.bike_trip_data.bikeshare_trips_clean` \
GROUP BY day_of_week \
ORDER BY trip_length_sum_per_week DESC; \
'

Waiting on bqjob_r1332d1a1a98ad355_00000168a1e5d31e_1 ... (0s) Current status: DONE   
+-------------+--------------------------+
| day_of_week | trip_length_sum_per_week |
+-------------+--------------------------+
|           6 |                135725265 |
|           3 |                133755198 |
|           4 |                132861324 |
|           5 |                132638530 |
|           2 |                129606217 |
|           7 |                115718564 |
|           1 |                105157765 |
+-------------+--------------------------+


Day of week starts with Sunday as "1" and ends with Saturday as "7"
Clearly, there is less traffic during weekends than weekdays. So it makes sense to incentivise more people to use the service during weekends

#### Question 2
What are the top 10 commutes on a weekday? (Advertize the top 5. Provide extra incentives for the next 5)

In [27]:
! bq query --use_legacy_sql=false ' \
SELECT trip_name, COUNT(trip_name) as trip_frequency_weekdays \
FROM ( \
   SELECT CONCAT(start_station_name, "_", end_station_name) as trip_name, \
   EXTRACT (DAYOFWEEK from start_date) as day_of_week \
   FROM `reliable-jet-228900.bike_trip_data.bikeshare_trips_clean` \
   ORDER BY trip_name \
   ) \
   where day_of_week > 1 and day_of_week < 7 \
GROUP BY trip_name \
ORDER BY trip_frequency_weekdays DESC \
LIMIT 10; \
'

Waiting on bqjob_r1d1eaa24dc26e9e5_00000168a1e8ad21_1 ... (1s) Current status: DONE   
+----------------------------------------------------------------------------------------+-------------------------+
|                                       trip_name                                        | trip_frequency_weekdays |
+----------------------------------------------------------------------------------------+-------------------------+
| San Francisco Caltrain 2 (330 Townsend)_Townsend at 7th                                |                    8047 |
| Harry Bridges Plaza (Ferry Building)_Embarcadero at Sansome                            |                    7072 |
| 2nd at Townsend_Harry Bridges Plaza (Ferry Building)                                   |                    7043 |
| Townsend at 7th_San Francisco Caltrain 2 (330 Townsend)                                |                    6465 |
| Embarcadero at Sansome_Steuart at Market                                               |    

The top 10 weekday commutes are clearly the ones that Ford Bikeshare should focus on to increase revenues. Perhaps the stations in these routes (ex: San Francisco Caltrain 2) are heavily used and subscribers might be willing to pay extra to reserve bikes and docking stations. Ford Bikeshare might want to add more bikes/docking stations if needed. Perhaps more work needs to be done to figure out if any of these stations are running out of bikes or docking stations.

#### Question 3
What are the top 10 commutes over the weekend? (Advertize the top 5. Monetize the first 5 some more (tournaments?))

In [35]:
! bq query --use_legacy_sql=false ' \
SELECT trip_name, COUNT(trip_name) as trip_frequency_weekends \
FROM ( \
   SELECT CONCAT(start_station_name, "_", end_station_name) as trip_name, \
   EXTRACT (DAYOFWEEK from start_date) as day_of_week \
   FROM `reliable-jet-228900.bike_trip_data.bikeshare_trips_clean` \
   ORDER BY trip_name \
   ) \
   where day_of_week = 1 or day_of_week = 7 \
GROUP BY trip_name \
ORDER BY trip_frequency_weekends DESC \
LIMIT 10; \
'

Waiting on bqjob_rbc68f29dd2412e8_00000168a22f5a5d_1 ... (0s) Current status: DONE   
+---------------------------------------------------------------------------+-------------------------+
|                                 trip_name                                 | trip_frequency_weekends |
+---------------------------------------------------------------------------+-------------------------+
| Harry Bridges Plaza (Ferry Building)_Embarcadero at Sansome               |                    2078 |
| Embarcadero at Sansome_Harry Bridges Plaza (Ferry Building)               |                    1183 |
| Embarcadero at Sansome_Embarcadero at Sansome                             |                    1113 |
| Harry Bridges Plaza (Ferry Building)_Harry Bridges Plaza (Ferry Building) |                    1088 |
| Embarcadero at Bryant_Embarcadero at Sansome                              |                     697 |
| Embarcadero at Bryant_Harry Bridges Plaza (Ferry Building)                |     

Based on the above stats, Ford Bikeshare can advertize the top 5 routes so that other casual customers are interested in doing these trips over the weekend. Perhaps doing meetups to explore the area with bikes over the weekend is a good marketing campaign. Perhaps for the next 5 routes, Ford Bikeshare can provide incentives such as discounted pricing to entice more casual customers/tourists to use their system

#### Question 4
Do subscribers use the bikeshare system more frequently on weekdays compared to customers? How about the weekend?

In [29]:
! bq query --use_legacy_sql=false ' \
SELECT subscriber_type, COUNT(trip_id) as trip_count_weekdays \
FROM ( \
   SELECT subscriber_type, trip_id, EXTRACT (DAYOFWEEK from start_date) as day_of_week \
   FROM `reliable-jet-228900.bike_trip_data.bikeshare_trips_clean` \
   ) \
WHERE day_of_week > 1 and day_of_week < 7 \
GROUP BY subscriber_type \
ORDER BY trip_count_weekdays; \
'

Waiting on bqjob_r7ca43cb4f257995d_00000168a1ef5269_1 ... (1s) Current status: DONE   
+-----------------+---------------------+
| subscriber_type | trip_count_weekdays |
+-----------------+---------------------+
| Customer        |               80992 |
| Subscriber      |              783937 |
+-----------------+---------------------+


In [30]:
! bq query --use_legacy_sql=false ' \
SELECT subscriber_type, COUNT(trip_id) as trip_count_weekends \
FROM ( \
   SELECT subscriber_type, trip_id, EXTRACT (DAYOFWEEK from start_date) as day_of_week \
   FROM `reliable-jet-228900.bike_trip_data.bikeshare_trips_clean` \
   ) \
WHERE day_of_week = 1 or day_of_week = 7 \
GROUP BY subscriber_type \
ORDER BY trip_count_weekends; \
'

Waiting on bqjob_r405435a70c5a4676_00000168a1f04d1e_1 ... (1s) Current status: DONE   
+-----------------+---------------------+
| subscriber_type | trip_count_weekends |
+-----------------+---------------------+
| Customer        |               54613 |
| Subscriber      |               55833 |
+-----------------+---------------------+


Subscribers are very active over weekdays compared to customers. Clearly, it makes sense for Ford Bikeshare to offer incentives to lure more customers to become subscribers. But it is possible that subscribers are really the ones who use bikeshare to commute to work. And that customers are casual users (such as tourists). So incentivising customers vs subscribers to use bikeshare more frequently would require some more thought.

#### Question 5
Do customers/subscribers take longer trips over the weekend? (This is related to providing incentives over the weekend)

In [33]:
! bq query --use_legacy_sql=false ' \
SELECT subscriber_type, AVG(duration_sec) \
FROM ( \
   SELECT subscriber_type, duration_sec, EXTRACT (DAYOFWEEK from start_date) as day_of_week \
   FROM `reliable-jet-228900.bike_trip_data.bikeshare_trips_clean` \
   ) \
WHERE day_of_week > 1 and day_of_week < 7 \
GROUP BY subscriber_type; \
'


Waiting on bqjob_r39c9518bffc06d84_00000168a1f42dbc_1 ... (0s) Current status: DONE   
+-----------------+-------------------+
| subscriber_type |        f0_        |
+-----------------+-------------------+
| Customer        | 2740.884803437355 |
| Subscriber      | 564.5820926936767 |
+-----------------+-------------------+


In [34]:
! bq query --use_legacy_sql=false ' \
SELECT subscriber_type, AVG(duration_sec) \
FROM ( \
   SELECT subscriber_type, duration_sec, EXTRACT (DAYOFWEEK from start_date) as day_of_week \
   FROM `reliable-jet-228900.bike_trip_data.bikeshare_trips_clean` \
   ) \
WHERE day_of_week = 1 or day_of_week = 7 \
GROUP BY subscriber_type; \
'


Waiting on bqjob_r47d3686ad36ce550_00000168a1f5699f_1 ... (1s) Current status: DONE   
+-----------------+-------------------+
| subscriber_type |        f0_        |
+-----------------+-------------------+
| Customer        | 3390.846098914152 |
| Subscriber      | 639.2644314294431 |
+-----------------+-------------------+


On an average, trips are definitely longer during weekends than weekdays. Both customers and subscribers take longer trips during weekends, though clearly customers take trips that are 5x longer than subscribers on both weekdays and weekends.